In [24]:
# import SharedArray as sa
# sa.delete("shm://{}.grasp_img".format('indy7'))
# sa.delete("shm://{}.arm_img".format('indy7'))
# sa.delete("shm://{}.rh_vals".format('indy7'))
# sa.delete("shm://{}.result".format('indy7'))
# sa.delete("shm://{}.query_in".format('indy7'))
# sa.delete("shm://{}.response_out".format('indy7'))
# sa.delete("shm://{}.query_quit".format('indy7'))

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "filters")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = True
CLEARANCE = 1e-3

ROBOT_TYPE = RobotType.indy7
ROBOT_NAME = "indy0"
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]
TOOL_XYZ = (0,0,0.14)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.05
HOME_POSE = (0,0,0,0,0,0)

# ROBOT_TYPE = RobotType.panda
# ROBOT_NAME = "panda0"
# TOOL_LINK = "panda0_hand"
# TOOL_LINK_BUNDLE = ["panda0_hand", "panda0_link6"]
# TOOL_XYZ = (0,0,0.112)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.03
# HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)


assert TOOL_LINK_BUNDLE[0] == TOOL_LINK, "TOOL_LINK_BUNDLE should be in reverse order including actor's link as the first item"

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

connection_list
[False]


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", rname=ROBOT_NAME, _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
checkers_all = [gcheck, rcheck, lcheck]

In [12]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

## ui

In [13]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [14]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

##
# @class ObstacleBase
# @brief base class for obstacle generators
class ObstacleBase:
    RTH_MIN = None ## R: center ~ nearest point
    RTH_MAX = None
    RPY_MIN = None
    RPY_MAX = None
    DIM_MIN = None
    DIM_MAX = None
    GTYPE = None
    COLOR = (0.7,0.7,0.7,1)
    
    def __init__(self, gscene, name, sampler=np.random.uniform, DIM=None, RTH=None, RPY=None):
        self.name = name
        self.DIM = sampler(self.DIM_MIN, self.DIM_MAX) if DIM is None else DIM
        self.RTH = sampler(self.RTH_MIN, self.RTH_MAX) if RTH is None else RTH
        self.RPY = sampler(self.RPY_MIN, self.RPY_MAX) if RPY is None else RPY
        self.RPY[2] += self.RTH[1]
        self.XYZ = np.array(cyl2cart(*self.RTH))
        verts_rotated = np.matmul(Rot_rpy(self.RPY), (DEFAULT_VERT_DICT[self.GTYPE]*self.DIM).transpose())
        xy_normed = self.XYZ[:2]/(np.linalg.norm(self.XYZ[:2])+1e-6)
        verts_r_compo = np.dot(xy_normed, verts_rotated[:2,:])
        self.XYZ[:2] -= xy_normed[:2]*np.min(verts_r_compo)
        self.RTH[0] -= np.min(verts_r_compo)
        self.geometry = gscene.create_safe(gtype=self.GTYPE, name=self.name, link_name="base_link", 
                                  dims=self.DIM, center=tuple(self.XYZ), rpy=self.RPY, 
                                  color=self.COLOR, display=True, collision=True, fixed=True)
        self.subgeo_list = []
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_me, raddii_me = self.geometry.get_vertice_radius()
        verts_me_global = np.add(np.matmul(verts_me, self.geometry.orientation_mat.transpose()), 
                                 self.geometry.center)
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_me_global))-radii-raddii_me < 1e-4
        
##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,1)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        if floor_height > self.RTH_MIN[2]:
            self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.4
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
            
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    COLOR =  (0.2,0.2,0.8,1)
    H_RANGE = (0.4, 0.4)
    THICKNESS = 0.05
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, -0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Ceiling
# @brief Ceiling - upperbound of the workspace
class Ceiling(ObstacleBase):
    RTH_MIN = (0.0, 0, 1.0)
    RTH_MAX = (0.0, 0, 2)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Wall
# @brief define horizontal boundary of the workspace
class Wall(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (2.0, np.pi, 0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 6, 3)
    DIM_MAX = (0.1, 6, 3)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.3, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.1, 0.1, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.05, 0.1, 0.2)
    DIM_MAX = (0.05, 0.1, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.8,0.2,1)
    def __init__(self, gscene, name, workplane, XYZ_LOC=None, **kwargs):
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        
def clear_class(gscene, key, Nmax):
    for iw in range(Nmax):
        gname = "{}_{}".format(key, iw)
        if gname in gscene.NAME_DICT:
            gscene.remove(gscene.NAME_DICT[gname])

            
def redistribute_class(gscene, obstacle_class, key, Nmax, workplane_avoid=None):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    for iw in range(np.random.choice(Nmax)):
        obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        while workplane_avoid is not None and workplane_avoid.is_overlapped_with(obs.geometry):
            obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        obs_list.append(obs)
    return obs_list

            
def disperse_objects(gscene, object_class, key, Nmax, workplane_on):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    while len(obs_list)<Nmax:
        iw = len(obs_list)
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2-CLEARANCE], [0,0,0])}
    action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
    obj_pscene = pscene.create_object(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles

   Use a production WSGI server instead.
 * Debug mode: off


In [15]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/filters/indy7/20210225-034545



### sampling 3-obj

In [16]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_shelf = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [17]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_shelf[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->1 (45.27/600.0 s, steps/err: 30(60.5561733246 ms)/0.00163275484728)
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->2 (50.37/600.0 s, steps/err: 43(55.8791160583 ms)/0.0017119313334)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 2->3 (50.39/600.0 s, steps/err: 6(25.260925293 ms)/0.00173109958115)
node: ('grip0', 'wp', 'wp')->('gp

node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 5->13 (498.77/600.0 s, steps/err: 12(28.8510322571 ms)/0.00192360085582)
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 

node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 3->7 (1.14/600.0 s, steps/err: 43(245.893955231 ms)/0.00140351130616)
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 0->8 (1.17/600.0 s, steps/err: 17(26.3998508453 ms)/0.00168196946543)
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = success
branching: 5->9 (1.48/600.0 s, steps/err: 37(310.570001602 ms)/0.00140385251617)
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->10 (1.59/600.0 s, steps/err: 77(104.422807693 ms)/0.00133938388249)
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
nod

node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 23->25 (52.98/600.0 s, steps/err: 21(50.0960350037 ms)/0.0021238475502)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->26 (53.01/600.0 s, steps/err: 24(29.6678543091 ms)/0.00143061124764)
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
nod

node: ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
node: ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp') = success
branching: 7->44 (85.02/600.0 s, steps/err: 27(40.2138233185 ms)/0.00124397424701)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'gp', 'grip0') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'grip0', 'gp') = success
branching: 31->45 (95.14/600.0 s, steps/err: 49(55.1190376282 ms)/0.00112244025052)
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = success
branching: 31->46 (95.21/600.0 s, steps/err: 40(74.4671821594 ms)/0.00146036244944)
node: ('gp', 'wp', 'gp')-

...

node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = success
branching: 79->165 (146.3/600.0 s, steps/err: 22(220.155000687 ms)/0.00113742022931)
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('gp', 'gp', 'wp')->('gp', 'gp', 'grip0') = success
branching: 151->166 (146.39/600.0 s, steps/err: 20(50.274848938 ms)/0.00107035967086)
node: ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'gp')->('wp', 'gp', 'gp') = success
branching: 144->167 (147.28/600.0 s, steps/err: 61(887.540102005 ms)/0.00141036296896)
node: ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = success
branching: 122->168 (147.37/600.0 s, steps/err: 37(84.3291282654 ms)/0.00198969471284)
node: ('gp', 'gp', 'wp')->('gp', 'gp', 'grip0') = success
branching: 163->169 (147.42/600.0 s

node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 116->183 (166.11/600.0 s, steps/err: 40(169.075012207 ms)/0.00133991597141)
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = fail
node: ('gp', 'wp', 'grip0')->('gp', 'wp', 'wp') = fail
node: ('gp', 'wp', 'grip0')->('gp', 'wp', 'gp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'gp')->('wp', 'wp', 'gp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'grip0', 'gp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
node: ('gp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 160->184 (166.24/600.0 s, steps/err: 50(105.475187302 ms)/0.000937157523553)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'gp', 'grip0') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') =

node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->3 (5.17/600.0 s, steps/err: 12(23.2329368591 ms)/0.00185962282045)
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->4 (5.22/600.0 s, steps/err: 19(34.4531536102 ms)/0.00190305106675)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: (

node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 5->15 (51.87/600.0 s, steps/err: 5(22.4301815033 ms)/0.00201116837373)
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 2->16 (56.97/600.0 s, steps/err: 24(55.5939674377 ms)/0.00164243193863)
node

node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->3 (0.21/600.0 s, steps/err: 41(64.7070407867 ms)/0.00172249259866)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->4 (0.27/600.0 s, steps/err: 11(53.7900924683 ms)/0.00173072152612)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
bran

In [18]:
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

0: [201.17075272401175, 2.6666666666666665, 49.333333333333336, 39.833333333333336, 4, 10]
1: [154.4924559354782, 6.2, 85.0, 29.3, 0, 10]
2: [26.667377734184264, 3.8, 26.6, 4.9, 0, 10]
3: [28.016021728515625, 8.4, 62.2, 3.7, 0, 10]


In [19]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

## sample with pole

In [20]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_pole = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)

    pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])

    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(1,4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_pole[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

In [22]:
for k,v in results_dict_3_pole.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

1: [208.52604667345682, 4.666666666666667, 90.44444444444444, 40.0, 1, 10]
2: [137.5783672809601, 6.0, 100.1, 25.3, 0, 10]
3: [42.91876456472609, 4.222222222222222, 62.0, 6.555555555555555, 1, 10]


In [23]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_pole.json"), results_dict_3_pole)